In [57]:
import gurobipy as gp
import numpy as np

In [58]:
distance_to_bricks = np.loadtxt('brick_rp_distances.csv', delimiter=',', skiprows=1)
distance_to_bricks = distance_to_bricks[:,1:]
print(distance_to_bricks.shape)
distance_to_bricks

(22, 4)


array([[16.16, 24.09, 24.29, 21.12],
       [19.  , 26.47, 27.22, 17.33],
       [25.29, 32.5 , 33.41, 12.25],
       [ 0.  ,  7.94,  8.29, 36.12],
       [ 3.07,  6.45,  7.55, 37.37],
       [ 1.22,  7.51,  8.17, 36.29],
       [ 2.8 , 10.3 , 10.92, 33.5 ],
       [ 2.87,  5.07,  5.65, 38.8 ],
       [ 3.8 ,  8.02,  7.37, 38.16],
       [12.35,  4.51,  4.37, 48.27],
       [11.11,  3.51,  2.97, 47.14],
       [21.99, 22.01, 24.1 , 39.86],
       [ 8.82,  3.28,  5.38, 43.31],
       [ 7.94,  0.  ,  2.1 , 43.75],
       [ 9.34,  2.32,  1.11, 45.43],
       [ 8.29,  2.1 ,  0.  , 44.4 ],
       [ 7.28,  2.4 ,  1.08, 43.4 ],
       [ 7.55,  0.77,  1.54, 43.52],
       [11.13, 18.41, 19.24, 25.4 ],
       [17.49, 23.44, 24.76, 23.21],
       [11.03, 18.94, 19.26, 25.43],
       [36.12, 43.75, 44.4 ,  0.  ]])

In [59]:
index_values = np.loadtxt('bricks_index_values.csv', delimiter=',', skiprows=1)
index_values = index_values[:,1:]
index_values

array([[0.1609],
       [0.1164],
       [0.1026],
       [0.1516],
       [0.0939],
       [0.132 ],
       [0.0687],
       [0.093 ],
       [0.2116],
       [0.2529],
       [0.0868],
       [0.0828],
       [0.0975],
       [0.8177],
       [0.4115],
       [0.3795],
       [0.071 ],
       [0.0427],
       [0.1043],
       [0.0997],
       [0.1698],
       [0.2531]])

In [61]:

# Instanciation du modèle

def optimize_distance1(distance_to_bricks, index_values, workload_range=[0.8, 1.2]):
    '''try to minimize the total sum of the distances'''
    m = gp.Model("Simple PL modelling")
    # Création de 2 variables continues v0 et v1
    v = m.addMVar(shape=distance_to_bricks.shape, vtype=gp.GRB.BINARY, name="v")

    m.update()

    m.addConstr( index_values.T @ v <= np.ones((1, distance_to_bricks.shape[1])) * workload_range[1] )
    m.addConstr( index_values.T @ v >= np.ones((1, distance_to_bricks.shape[1])) * workload_range[0] )
    m.addConstr( v @ np.ones(distance_to_bricks.shape[1]) == np.ones(distance_to_bricks.shape[0]) )

    m.update()

    m.setObjective( v * distance_to_bricks @ np.ones(distance_to_bricks.shape[1]) @ np.ones(distance_to_bricks.shape[0]) , gp.GRB.MINIMIZE)

    m.update()
        
    m.params.outputflag = 0
    # # Résolution du PL
    m.optimize()
    return v.X

print(optimize_distance1(distance_to_bricks, index_values))

[[-0. -0. -0.  1.]
 [-0. -0. -0.  1.]
 [-0. -0. -0.  1.]
 [ 1. -0. -0. -0.]
 [ 1. -0. -0. -0.]
 [ 1. -0. -0. -0.]
 [ 1. -0. -0. -0.]
 [ 1. -0. -0. -0.]
 [ 1. -0. -0. -0.]
 [-0. -0.  1. -0.]
 [-0.  1.  0. -0.]
 [ 1. -0. -0. -0.]
 [-0.  1. -0. -0.]
 [-0.  1. -0. -0.]
 [-0. -0.  1. -0.]
 [-0. -0.  1. -0.]
 [-0.  0.  1. -0.]
 [-0.  1.  0. -0.]
 [ 1. -0. -0. -0.]
 [ 1. -0. -0.  0.]
 [-0. -0. -0.  1.]
 [-0. -0. -0.  1.]]
